In [1]:
from datetime import datetime, timedelta
from decimal import Decimal
from pprint import pprint

from analyst.adapters.factory import get_adapters
from analyst.bot.order_manager import OrderManager, Side, PairSide
from analyst.controllers.factory import get_controllers
from analyst.repositories.factory import get_repositories
from analyst.settings import get_settings

In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
settings = get_settings()
adapters = await get_adapters(settings=settings)
repositories = get_repositories(
    settings=settings,
    adapters=adapters
)
controllers = get_controllers(
    adapters=adapters,
    repositories=repositories
)

order_manager = OrderManager(controllers=controllers)

await order_manager.setup()

In [4]:
btcusdt = order_manager.pairs["BTCUSDT"]

pprint(btcusdt.dict())

{'ask_price': Decimal('19140.34000000'),
 'ask_quantity': Decimal('0.01192000'),
 'base': 'BTC',
 'base_asset_precision': 8,
 'bid_price': Decimal('19139.59000000'),
 'bid_quantity': Decimal('0.00517000'),
 'max_quantity': Decimal('9000.0'),
 'min_quantity': Decimal('0.00001'),
 'quote': 'USDT',
 'quote_asset_precision': 8,
 'quote_min_amount': Decimal('10.0'),
 'step_size': Decimal('0.00001')}


In [163]:
df = await controllers.binance.get_klines(
    symbol=btcusdt.symbol,
    interval="5m",
    start_datetime=datetime.now() - timedelta(days=180),
)

In [164]:
quote_value = Decimal("10")
interval = Decimal("50")
max_buy_orders = 3

buy_orders = {}
sell_orders = {}

invested_quantity = Decimal()
sold_quantity = Decimal()

last_price = None
last_floored_price = None

def format_sort(array):
    return ",".join([str(i) for i in sorted(array, reverse=True)])

for price in df.close:#[:1390]:
    price = Decimal(str(price))
    
    floored_price = (price // interval) * interval
    
    converted = order_manager.truncate_base_quantity(
        btcusdt,
        order_manager.convert_quantity(quote_value, floored_price, to=PairSide.base),
        ceil=True
    )
    
    #for order_price, order_quantity in list(buy_orders.items()):
    
    floored_change = ""
    if last_floored_price:
        if floored_price > last_floored_price:
            floored_change = "up"
        elif floored_price < last_floored_price:
            floored_change = "down"
        
    sold = 0
    for order_price, order_quantity in list(sell_orders.items()):
        if order_price < floored_price:
            del sell_orders[order_price]
            sold_quantity += Decimal(order_quantity * order_price)
            sold += 1

    sell_back = 0
    for order_price, order_quantity in list(buy_orders.items()):
        if order_price > floored_price:
            sell_orders[order_price + interval] = order_quantity
            del buy_orders[order_price]
            sell_back += 1
    
    buy_created = 0
    for atom_index in range(max_buy_orders):
        buy_price = floored_price - (atom_index * interval)
        
        if buy_price not in buy_orders and buy_price + interval not in sell_orders:
            converted2 = order_manager.truncate_base_quantity(
                btcusdt,
                order_manager.convert_quantity(quote_value, buy_price, to=PairSide.base),
                ceil=True
            )
            buy_orders[buy_price] = converted2
            buy_created += 1
            invested_quantity += buy_price * converted2

    buy_invested = sum([
        price * quantity
        for price, quantity in buy_orders.items()
    ])
    sell_invested = sum([
        price * quantity
        for price, quantity in sell_orders.items()
    ])
    if 0:
        print(
            f"{price:5.2f} = "
            f"{floored_price:.4f} = "
            f"{converted} => "
            f"{converted * floored_price} - "
            f"{floored_change:4s} - "
            f"b={buy_created} s={sell_back} S={sold} - "
            f"buys={len(buy_orders)} - "
            f"sell={len(sell_orders)} - "
            f"sq={sold_quantity:10f} "
            f"bi={buy_invested:10f} "
            f"si={sell_invested:10f} "
            f"iq={invested_quantity:10f} "
            f"all={invested_quantity + sell_invested:10f} "
            f"ratio={sold_quantity / (invested_quantity + sell_invested) * 100:5.2f}% "
            #f"{invested_quantity}"
            #f"{invested=} "
        )
    
    last_price = price
    last_floored_price = floored_price

sell_invested2 = sum([
    last_price * quantity
    for price, quantity in sell_orders.items()
])  

if True:
    print(
        f"{price:5.2f} = "
        f"{floored_price:.4f} = "
        f"{converted} => "
        f"{converted * floored_price} - "
        f"{floored_change:4s} - "
        f"b={buy_created} s={sell_back} S={sold} - "
        f"buys={len(buy_orders)} - "
        f"sell={len(sell_orders)} - "
        f"sq={sold_quantity:10f} "
        f"bi={buy_invested:10f} "
        f"si={sell_invested:10f} "
        f"si2={sell_invested2:10f} "
        f"iq={invested_quantity:10f} "
        f"ratio={sold_quantity / invested_quantity * 100:5.2f}% "
        f"ratio={(sold_quantity + sell_invested2) / invested_quantity * 100:5.2f}% "
        #f"{invested_quantity}"
        #f"{invested=} "
    )

19179.14 = 19150.0000 = 0.00053 => 10.14950 -      - b=0 s=0 S=0 - buys=30 - sell=517 - sq=71755.27800 bi= 302.65850 si=5263.95850 si2=3309.9359812 iq=77176.30750 ratio=92.98% ratio=97.26% 


In [156]:
4191.54600 + 121.09000 + 609.47000

4922.106000000001

In [173]:
buy_created

0

In [89]:
0.00055 * 18150

9.9825

In [172]:
print((0.00055 * 18200) - (0.00055 * 18150))
print((0.00055 * 1050) - (0.00055 * 1000))
print(((0.00055 * 1050) - (0.00055 * 1000)) / 10 * 100)

0.027499999999999858
0.02749999999999997
0.2749999999999997


In [168]:
#buy_orders
#sell_orders
print()
print()
for price, quantity in buy_orders.items():
    print(price, quantity, price * quantity)
print()
for price, quantity in sell_orders.items():
    print(price, quantity, price * quantity)



17650 0.00057 10.06050
17600 0.00057 10.03200
17550 0.00057 10.00350
17700 0.00057 10.08900
17900 0.00056 10.02400
17850 0.00057 10.17450
18000 0.00056 10.08000
17950 0.00056 10.05200
18050 0.00056 10.10800
18200 0.00055 10.01000
18150 0.00056 10.16400
18100 0.00056 10.13600
18250 0.00055 10.03750
18300 0.00055 10.06500
18350 0.00055 10.09250
18400 0.00055 10.12000
18450 0.00055 10.14750
18500 0.00055 10.17500
18550 0.00054 10.01700
18600 0.00054 10.04400
18650 0.00054 10.07100
18750 0.00054 10.12500
18850 0.00054 10.17900
18800 0.00054 10.15200
18900 0.00053 10.01700
19050 0.00053 10.09650
19000 0.00053 10.07000
18950 0.00053 10.04350
19100 0.00053 10.12300
19150 0.00053 10.14950

45450 0.00023 10.45350
45500 0.00023 10.46500
45400 0.00023 10.44200
45300 0.00023 10.41900
45350 0.00023 10.43050
45250 0.00023 10.40750
45200 0.00023 10.39600
45150 0.00023 10.38450
45100 0.00023 10.37300
45050 0.00023 10.36150
45000 0.00023 10.35000
44950 0.00023 10.33850
44900 0.00023 10.32700
44850 0.

In [67]:
0.00050 * 20000

10.0

In [ ]:
a = Decimal(10 / 19748.05)
a

In [ ]:

pprint(btcusdt.dict())

In [15]:
quote_price = Decimal(10 / 19748.05)
print(f"{quote_price=}")
pprint(btcusdt.dict())
print(f"{btcusdt.step_size=}")

quotient, remainder = divmod(quote_price, btcusdt.step_size)

product = quotient * btcusdt.step_size
if remainder:
    product2 = product + btcusdt.step_size

print(f"{quotient=} {remainder=}")
print(f"{product=} {product2=}")




quote_price=Decimal('0.000506379110848919288499769830735885989270173013210296630859375')
{'ask_price': Decimal('19140.34000000'),
 'ask_quantity': Decimal('0.01192000'),
 'base': 'BTC',
 'base_asset_precision': 8,
 'bid_price': Decimal('19139.59000000'),
 'bid_quantity': Decimal('0.00517000'),
 'max_quantity': Decimal('9000.0'),
 'min_quantity': Decimal('0.00001'),
 'quote': 'USDT',
 'quote_asset_precision': 8,
 'quote_min_amount': Decimal('10.0'),
 'step_size': Decimal('0.00001')}
btcusdt.step_size=Decimal('0.00001')
quotient=Decimal('50') remainder=Decimal('0.000006379110848919288499769830736')
product=Decimal('0.00050') product2=Decimal('0.00051')
